## **Multiple-Runway Constraint Programming Model**

<h4 style="color:#0033a0">Parameters</h4>

\begin{aligned}
p 
& \quad \text{Number of aircraft (planes)}, \\[6pt]
r 
& \quad \text{Number of runways}, \\[6pt]
E_i 
& \quad \text{Earliest landing time for aircraft } i, \\[6pt]
T_i 
& \quad \text{Target (preferred) landing time for aircraft } i, \\[6pt]
L_i 
& \quad \text{Latest landing time for aircraft } i, \\[6pt]
cost_e^i 
& \quad \text{Penalty cost per unit time if } t_i < T_i \ (\text{earliness cost}), \\[6pt]
cost_l^i 
& \quad \text{Penalty cost per unit time if } t_i > T_i \ (\text{lateness cost}), \\[6pt]
sep_{i,j} 
& \quad \text{Separation time required if aircraft } i \text{ lands immediately before } j 
\text{ (only if on the same runway)}.
\end{aligned}


<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{aligned}
position_i 
& \quad \text{Position of aircraft } i \text{ in the landing sequence}, \\[6pt]
landing\_time_i 
& \quad \text{Landing time of aircraft } i, \\[6pt]
earliness_i 
& \quad \text{Earliness of aircraft } i \text{ (if } t_i < T_i \text{)}, \\[6pt]
lateness_i 
& \quad \text{Lateness of aircraft } i \text{ (if } t_i > T_i \text{)}, \\[6pt]
runway_i 
& \quad \text{Runway assigned to aircraft } i.
\end{aligned}


<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{aligned}
\text{Constraints:} \\[6pt]
& E_i \leq landing\_time_i \leq L_i \quad \forall i, \\[6pt]
& earliness_i \geq T_i - landing\_time_i \quad \forall i, \\[6pt]
& earliness_i \geq 0 \quad \forall i, \\[6pt]
& lateness_i \geq landing\_time_i - T_i \quad \forall i, \\[6pt]
& lateness_i \geq 0 \quad \forall i, \\[6pt]
& \text{If } runway_i = runway_j \text{ and } position_i < position_j, \text{ then } landing\_time_j \geq landing\_time_i + sep_{i,j} \quad \forall i, j.
\end{aligned}

\begin{aligned}
\text{Objective Function:} \\[6pt]
& \text{Minimize } \sum_{i=1}^{p} (cost_e^i \cdot earliness_i + cost_l^i \cdot lateness_i).
\end{aligned}

In [10]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []
            
            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )
                
                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None

In [11]:
filename = "data/airland4.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

if num_planes is not None:
    print("Number of planes:", num_planes)
    print("Freeze time:", freeze_time)
    print("\nPlane data:")
    for i, plane in enumerate(planes_data):
        print(f"Plane {i+1}: {plane}")
    print("\nSeparation times:")
    for i, row in enumerate(separation_times):
        print(f"After plane {i+1}: {row}")

Number of planes: 20
Freeze time: 35

Plane data:
Plane 1: {'appearance_time': 7, 'earliest_landing_time': 82, 'target_landing_time': 92, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 2: {'appearance_time': 9, 'earliest_landing_time': 84, 'target_landing_time': 93, 'latest_landing_time': 509, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 3: {'appearance_time': 75, 'earliest_landing_time': 150, 'target_landing_time': 183, 'latest_landing_time': 599, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 4: {'appearance_time': 129, 'earliest_landing_time': 204, 'target_landing_time': 270, 'latest_landing_time': 760, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 5: {'appearance_time': 14, 'earliest_landing_time': 89, 'target_landing_time': 98, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 6: {'appearance_time': 29, 'earliest_landing_time': 104, 'target_landing_time': 117, 'latest_landing_time': 552, 'penalty_early': 3

In [12]:
from ortools.sat.python import cp_model

def create_or_tools_cp_model_with_permutation(num_planes, num_runways, freeze_time, planes_data, separation_times):
    model = cp_model.CpModel()

    E = [p["earliest_landing_time"] for p in planes_data]
    T = [p["target_landing_time"] for p in planes_data]
    L = [p["latest_landing_time"] for p in planes_data]
    cost_e = [p["penalty_early"] for p in planes_data]
    cost_l = [p["penalty_late"] for p in planes_data]

    position = [model.NewIntVar(0, num_planes - 1, f"position_{i}") for i in range(num_planes)]
    model.AddAllDifferent(position)

    landing_time = [model.NewIntVar(0, 10000000, f"landing_time_{i}") for i in range(num_planes)]
    earliness = [model.NewIntVar(0, max(T[i] - E[i], 0), f"earliness_{i}") for i in range(num_planes)]
    lateness = [model.NewIntVar(0, max(L[i] - T[i], 0), f"lateness_{i}") for i in range(num_planes)]
    runway = [model.NewIntVar(0, num_runways - 1, f"runway_{i}") for i in range(num_planes)]

    for i in range(num_planes):
        model.Add(landing_time[i] >= E[i])
        model.Add(landing_time[i] <= L[i])
        model.Add(earliness[i] >= T[i] - landing_time[i])
        model.Add(earliness[i] >= 0)
        model.Add(lateness[i] >= landing_time[i] - T[i])
        model.Add(lateness[i] >= 0)

    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            iBeforeJ = model.NewBoolVar(f"iBeforeJ_{i}_{j}")
            model.Add(position[i] < position[j]).OnlyEnforceIf(iBeforeJ)
            model.Add(position[i] >= position[j]).OnlyEnforceIf(iBeforeJ.Not())

            same_runway = model.NewBoolVar(f"same_runway_{i}_{j}")
            model.Add(runway[i] == runway[j]).OnlyEnforceIf(same_runway)
            model.Add(runway[i] != runway[j]).OnlyEnforceIf(same_runway.Not())

            model.Add(landing_time[j] >= landing_time[i] + separation_times[i][j]).OnlyEnforceIf(iBeforeJ).OnlyEnforceIf(same_runway)
            model.Add(landing_time[i] >= landing_time[j] + separation_times[j][i]).OnlyEnforceIf(iBeforeJ.Not()).OnlyEnforceIf(same_runway)

    cost_terms = []
    for i in range(num_planes):
        cost_terms.append(cost_e[i] * earliness[i])
        cost_terms.append(cost_l[i] * lateness[i])
    model.Minimize(sum(cost_terms))

    variables = {
        "position": position,
        "landing_time": landing_time,
        "earliness": earliness,
        "lateness": lateness,
        "runway": runway
    }

    return model, variables

def solve_cp_model_with_permutation(num_planes, num_runways, freeze_time, planes_data, separation_times, search_strategy):
    model, vars_ = create_or_tools_cp_model_with_permutation(num_planes, num_runways, freeze_time, planes_data, separation_times)

    solver = cp_model.CpSolver()

    if search_strategy == "default":
        pass  # Use default search strategy
    elif search_strategy == "min_conflicts":
        solver.parameters.search_branching = cp_model.VarBranchingPolicy.MIN_CONFLICTS
    elif search_strategy == "first_fail":
        solver.parameters.search_branching = cp_model.VarBranchingPolicy.FIRST_FAIL
    elif search_strategy == "automatic":
        solver.parameters.search_branching = cp_model.VarBranchingPolicy.AUTOMATIC
    else:
        raise ValueError(f"Unknown search strategy: {search_strategy}")

    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("Status:", solver.StatusName(status))
        print("Objective (total earliness + lateness):", solver.ObjectiveValue())
        print()

        position = vars_["position"]
        landing_time = vars_["landing_time"]
        earliness = vars_["earliness"]
        lateness = vars_["lateness"]
        runway = vars_["runway"]

        for i in range(num_planes):
            pos_val = solver.Value(position[i])
            t_val = solver.Value(landing_time[i])
            e_ = solver.Value(earliness[i])
            L_ = solver.Value(lateness[i])
            T_ = planes_data[i]["target_landing_time"]
            r_ = solver.Value(runway[i])
            print(f"Plane {i}: position={pos_val}, landing={t_val}, runway={r_}, E'={e_}, L'={L_}, target={T_}")

        print("\nSchedule order (by position):")
        schedule = sorted(range(num_planes), key=lambda i: solver.Value(position[i]))
        for k in schedule:
            print(f" -> Plane {k} (pos={solver.Value(position[k])}, land={solver.Value(landing_time[k])}, runway={solver.Value(runway[k])})")

    else:
        print("No feasible/optimal solution found. Status:", solver.StatusName(status))

if __name__ == "__main__":
    num_runways = 5
    search_strategy = "min_conflicts"  # Example search strategy
    solve_cp_model_with_permutation(num_planes, num_runways, freeze_time, planes_data, separation_times, search_strategy)

Status: OPTIMAL
Objective (total earliness + lateness): 0.0

Plane 0: position=0, landing=92, runway=4, E'=0, L'=0, target=92
Plane 1: position=1, landing=93, runway=2, E'=0, L'=0, target=93
Plane 2: position=7, landing=183, runway=4, E'=0, L'=0, target=183
Plane 3: position=15, landing=270, runway=3, E'=0, L'=0, target=270
Plane 4: position=9, landing=98, runway=3, E'=0, L'=0, target=98
Plane 5: position=2, landing=117, runway=2, E'=0, L'=0, target=117
Plane 6: position=12, landing=118, runway=1, E'=0, L'=0, target=118
Plane 7: position=10, landing=116, runway=3, E'=0, L'=0, target=116
Plane 8: position=11, landing=112, runway=0, E'=0, L'=0, target=112
Plane 9: position=17, landing=280, runway=0, E'=0, L'=0, target=280
Plane 10: position=18, landing=295, runway=3, E'=0, L'=0, target=295
Plane 11: position=6, landing=156, runway=4, E'=0, L'=0, target=156
Plane 12: position=3, landing=137, runway=2, E'=0, L'=0, target=137
Plane 13: position=8, landing=291, runway=2, E'=0, L'=0, target=2